# Stochastic Unit Commitment Problem

In [19]:
import sys
sys.path.append('../sddip')

In [20]:
#%run ../scripts/create_result_directories.py

In [21]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg


import sddip.tree as tree
import sddip.storage as storage
import sddip.utils as utils
import sddip.config as config
import sddip.dualsolver as dualsolver
import sddip.ucmodel as ucmodel


## Data Processing

In [22]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")

scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

### Bus Data

In [23]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


### Branch Data

In [24]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


### Generator Data

In [25]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


### Generator Cost Data

In [26]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [27]:
scenario_df
# TODO Stochastische Nachfrage für jeden Knoten

,t,n,p,Pd
0,1,1,1.0,350
1,2,1,0.5,300
2,2,2,0.5,200


### Power Transfer Distribution Factor

In [28]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1,ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus-1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

### Parameter Organization

In [29]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

probs = np.empty(n_stages, dtype=object)
# p_d[t,n]
p_d = np.empty(n_stages, dtype=object)

for t in range(n_stages):
    stage_df = scenario_df.loc[scenario_df["t"] == t+1]
    probs[t] = np.array(stage_df.p)
    p_d[t] = np.array(stage_df.Pd)

########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = np.array([probs[t].dot(p_d[t]) for t in range(n_stages)])

In [30]:
probs

array([array([1.]), array([0.5, 0.5])], dtype=object)

## SDDiP

In [31]:
# Result keys
x_key = "x"
y_key = "y"
z_key = "z"
primal_solution_keys = [x_key, y_key, z_key]

dv_key = "dual_value"
dm_key = "dual_multiplier"
dual_solution_keys = [dv_key, dm_key]

ci_key = "intercepts"
cg_key = "gradients"
bm_key = "multipliers"
cut_coefficient_keys = [ci_key, cg_key, bm_key]


# Result storage
ps_storage = storage.ResultStorage(primal_solution_keys)
ds_storage = storage.ResultStorage(dual_solution_keys)
cc_storage = storage.ResultStorage(cut_coefficient_keys)


### Forward pass

In [32]:
penalty = 10000
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist() + [penalty]*2
present_demand = p_d[0][0]
theta_lb = 0
trial_point = [0]*n_gens


# TODO Sampling
samples = [[0,1], [0,0]]
n_samples = len(samples)

v_opt_k = []
i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    for t, n in zip(range(n_stages), samples[k]):

        # Create forward model
        uc_fw = ucmodel.ForwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

        uc_fw.add_objective(cost_coeffs)

        uc_fw.add_balance_constraints(p_d[t][n])

        uc_fw.add_power_flow_constraints(ptdf, pl_max, [0,p_d[t][n]])

        uc_fw.add_generator_constraints(pg_min, pg_max)

        uc_fw.add_startup_shutdown_constraints()

        uc_fw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

        uc_fw.add_copy_constraints(trial_point)

        #TODO Lower bound
        uc_fw.add_cut_lower_bound(theta_lb)
        
        if i>0:
            cut_coefficients = cc_storage.get_stage_results[t]
            uc_fw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])
        
        # Solve problem
        uc_fw.suppress_output()
        uc_fw.model.optimize()
        uc_fw.model.printAttr("X")

        # Store xtik, ytik, ztik, vtik
        y_kt = [y_g.x for y_g in uc_fw.y]
        z_kt = [z_g.x for z_g in uc_fw.z]
        s_up_kt = [s_up_g.x for s_up_g in uc_fw.s_up]
        s_down_kt = [s_down_g.x for s_down_g in uc_fw.s_down]
        
        # Value of stage t objective function
        v_opt_kt = uc_fw.model.getObjective().getValue() - uc_fw.theta.x

        v_opt_k[-1] += v_opt_kt

        # New trial point
        trial_point = [x_g.x for x_g in uc_fw.x]

        ps_dict = ps_storage.create_empty_result_dict()
        ps_dict[x_key] = trial_point
        ps_dict[y_key] = y_kt
        ps_dict[z_key] = z_kt
        
        ps_storage.add_result(i, k, t, ps_dict)

#### Statistical Upper Bound

In [33]:
v_opt_k = np.array(v_opt_k)

v_mean = np.mean(v_opt_k)
v_std = np.std(v_opt_k)
alpha = 0.05

v_upper = v_mean + stats.norm.ppf(alpha/2)*v_std/np.sqrt(n_samples)


In [34]:
primal_solution_df = ps_storage.to_dataframe()
primal_solution_df

x        y      z
i k t                       
0 0 0  [1.0]  [350.0]  [0.0]
    1  [1.0]  [200.0]  [1.0]
  1 0  [1.0]  [350.0]  [1.0]
    1  [1.0]  [300.0]  [1.0]

### Backward pass

In [35]:
########################################################################################################################
# Backward pass
########################################################################################################################


binarizer = utils.Binarizer()

sg_method = dualsolver.SubgradientMethod(max_iterations=100)

cut_lower_bounds = [theta_lb]*n_stages
samples = [[0,1], [0,0]]
n_samples = len(samples)
v_opt_k =[]


i = 0
for k in range(n_samples):
    v_opt_k.append(0)
    for t in reversed(range(n_stages)):
        n_realizations = n_nodes_per_stage[t]
        ds_dict = ds_storage.create_empty_result_dict()
        cc_dict = cc_storage.create_empty_result_dict()
        
        for n in range(n_realizations):
            
            # TODO Binarization
            bin_vars = []
            bin_multipliers = []
            if t>0: 
                float_vars = ps_storage.get_result(i,k,t-1)[y_key]
            else:
                #TODO Approximation needed?
                float_vars = np.zeros(len(float_vars))
            for j in range(len(float_vars)):
                new_vars, new_multipliers = binarizer.binary_expansion(float_vars[j], upper_bound=pg_max, precision=0.5)
                bin_vars += new_vars
                bin_multipliers.append(new_multipliers) 


            binary_trial_point = bin_vars
            binary_trial_multipliers = linalg.block_diag(*bin_multipliers)


            uc_bw = ucmodel.BackwardModelBuilder(n_buses, n_lines, n_gens, gens_at_bus)

            uc_bw.add_objective(cost_coeffs)

            uc_bw.add_balance_constraints(p_d[t][n])

            uc_bw.add_generator_constraints(pg_min, pg_max)

            uc_bw.add_power_flow_constraints(ptdf, pl_max, [0, p_d[t][n]])

            uc_bw.add_startup_shutdown_constraints()

            uc_bw.add_ramp_rate_constraints(rg_up_max, rg_down_max)

            uc_bw.add_relaxation(binary_trial_point)

            uc_bw.add_copy_constraints(binary_trial_multipliers)

            uc_bw.add_cut_lower_bound(cut_lower_bounds[t])
            
            if t < n_stages-1:
                cut_coefficients = cc_storage.get_stage_result(t)
                uc_bw.add_cut_constraints(cut_coefficients[ci_key], cut_coefficients[cg_key], cut_coefficients[bm_key])

            objective_terms = uc_bw.objective_terms
            relaxed_terms = uc_bw.relaxed_terms

            uc_bw.suppress_output()

            model, sg_results = sg_method.solve(uc_bw.model, objective_terms, relaxed_terms, 10000)
            model.printAttr("X")       
            
            # Dual value and multiplier for each realization
            ds_dict[dv_key].append(sg_results.obj_value)
            ds_dict[dm_key].append(sg_results.multipliers)
            
        
        ds_storage.add_result(i, k, t, ds_dict)

        # Calculate and store cut coefficients
        probabilities = probs[t]        
        intercept = np.array(probabilities).dot(np.array(ds_dict[dv_key]))
        gradient = np.array(probabilities).dot(np.array(ds_dict[dm_key]))

        cc_dict[ci_key] = intercept.tolist()
        cc_dict[cg_key] = gradient.tolist()
        cc_dict[bm_key] = binary_trial_multipliers

        if t > 0 : cc_storage.add_result(i, k, t-1, cc_dict)
                

In [36]:
# One dual value and one set of dual multipliers for each realization
ds_storage.to_dataframe()

dual_value  \
i k t                                           
0 0 1  [16579.99390411609, 16719.99377441645]   
    0                               [10000.0]   
  1 1  [16579.99390411609, 16719.99377441645]   
    0                               [10000.0]   

                                         dual_multiplier  
i k t                                                     
0 0 1  [[1879.9992828371876, 1879.999282837187, 1879....  
    0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
  1 1  [[1879.9992828371876, 1879.999282837187, 1879....  
    0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

In [37]:
cc_storage.to_dataframe()

intercepts                                          gradients  \
i k t                                                                    
0 0 0  16649.993839  [1899.999275207796, 1899.999275207796, 1899.99...   
  1 0  16649.993839  [1899.999275207796, 1899.999275207796, 1899.99...   

                                             multipliers  
i k t                                                     
0 0 0  [[0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 1...  
  1 0  [[0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 1...